# Cable Robot Calibration

This notebook calibrates the cable robot by recording all the motor angles as you move around the robot end effector by hand.  It then fits a model to the data and writes the parameters to the cable robot EEPROM.

In [ ]:
%load_ext autoreload
%autoreload 1
%aimport communicate
from communicate import CableRobot, MotorState, ControllerState
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy
import scipy.optimize

## Data Collection

First, we need to collect data of the robot being manually moved around.

In [ ]:
def collect_data(duration=5):
    """Collects log data from the cable robot.  Returns tuple of Nx4 arrays for lengths and vels"""
    # with CableRobot(print_raw=True, write_timeout=None, initial_msg='d10,1', port='/dev/tty.usbmodem100994303') as robot:
    with CableRobot(print_raw=True, write_timeout=None, initial_msg='d10,1', port='/dev/tty.usbmodem103568503') as robot:
        tstart = time.time()
        while True:
            robot.update()
            time.sleep(0.01)
            if (time.time() - tstart > duration):
                break
        robot.send('d10,100')
    motor_ls = np.array([[m.length for m in datum['motors']] for datum in robot.all_data])
    motor_ldots = np.array([[m.lengthdot for m in datum['motors']] for datum in robot.all_data])
    xy = np.array([[dat['controller'].cur_x, dat['controller'].cur_y] for dat in robot.all_data])
    return motor_ls, motor_ldots, xy

Test the connection by running the following cell.  If it throws an error saying:
* "Resource busy" - you probably connected to the wrong serial port in the HTML control panel.  Try manually connecting (un-select Auto-connect) or switching `AUTOCONNECT_INDEX` in `serial.js`.
* "No such file or directory" - The port name is wrong.  Find the name of the "second" serial port of the Teensy e.g. with `ls /dev/tty.*` or Arduino > Port or device manager or something.

In [ ]:
# Test connection
ls, ldots, xys = collect_data(duration=0.1)
print(ls.shape, ldots.shape, xys.shape)

Assuming the connection is good, run the following cell to collect 45 seconds worth of data.  While the cell is running, manually move the robot around by hand all around the workspace.

In [ ]:
if True:
    # Collect some data
    time.sleep(2)
    ls, ldots, xy = collect_data(duration=45)
    print(f'Collected {len(ls)} samples')
    print(ls[-1])
    # Save/Load data (in case of ipynb failure)
    np.savez('/tmp/data.npz', ls=ls, ldots=ldots, est=xy)
else:
    with np.load('/tmp/data.npz') as f:
        ls = f['ls']
        ldots = f['ldots']
        xy = f['est']

Make a backup just in case, then plot the data as a sanity check.
Hopefully, the right-most subplot looks roughly like the path you moved the robot.  If it is very different, then you'll have to try troubleshooting by:
1. translating/scaling the path so it seems more correct relative to the robot/frame origin (bottom left)
<!-- 2. Manually adjusting the cable zeros by manually measuring (e.g. with a tape measure) the length of each cable and typing `c30,xxx` where xxx is the length of cable 0, `c31,yyy` for cable 1, ..., `c33,zzz` for cable 3.  Also do step 3, then, re-run the data collection. -->
2. Manually adjusting the cable calibration parameters.  Try something like this:
  `c44,0,1,w,0,1,x,0,1,y,0,1,z`.  This is assuming the diameter of the drum is 1inch and there is no "cable-doubling" to get additional pulley advantage.  If there is, adjust the "1" accordingly (e.g. 0.5 if drum diameter is 0.5" and 0.25 if the cables are also doubled).  The `wxyz` are the lengths of the 4 cables at the zero position - measure this manually with a tape measure (or estimate based on frame dimensions).  Then, re-run the data collection.
3. Manually adjusting the frame calibration parameters.  Something like this: `c54,2.66,0.0,2.66,2.00,0.0,2.00,0.0,0.0` would correspond to a frame that is 2.66m wide and 2.00m tall, with the origin at the bottom left corner.  Again, re-run the data collection.


In [ ]:
xy_bak = xy.copy()

In [ ]:
# Manual adjustments
# xy = xy_bak - [0.0, 0.2]

In [ ]:
# Plot data as a sanity check
fig, axes = plt.subplots(1, 3, sharex=False, figsize=(12, 4))
axes[0].plot(ls)
axes[1].plot(ldots);
axes[2].plot(*xy.T)
axes[2].axis('equal')
axes[0].set_title('Cable Lengths (m)')
axes[1].set_title('Cable Velocities (m/s)')

## Calibration

This runs an optimization problem to try to compute the cable length parameters.

Make sure to set `W, H = ` to the appropriate frame dimensions minus the end-effector dimensions.  e.g. if the frame is 3m wide but the end-effector is 0.5m, then the effective width should only be `W = 2.5`.

This cell shouldn't take more than 10-15s to run.  If it's taking longer than that, it will probably fail.  Try the troubleshooting steps above.

In [ ]:
# Constants
# W, H = 2.75, 2.17
# W, H = 3.05 - .22377, 2.44 - .22377
# W, H = 3.05, 2.44
# W, H = 3.05, 2.34
# W, H = 2.2 - .22377, 2.0 - .22377
# W, H = 3.05 - 0.1778, 2.34 - 0.14
# W, H = 2.9 - 0.184, 2.26 - 0.122
# W, H = 2.92, 2.26
# W, H = 2.92 - 0.18, 2.26 - 0.18
W, H = 2.92 - 0.482, 2.26 - 0.482  # ME Capstone Arm
# W, H = 2.92, 2.26  # ME Capstone Arm
INIT_XS = lambda ls: np.ones(ls.shape[0] * 2) * 1.5
# INIT_LPARAMS = lambda ls: np.array([0,0,0,0,1,1,1,1,*(-ls.mean(axis=0) + 1.5)])
# INIT_LPARAMS = lambda ls: np.array([0,0,0,0,1,1,1,1,*(-ls.mean(axis=0) + 1.5)])
init_ls = [1.1, 2.0, 2.0, 1.1]
INIT_LPARAMS = lambda ls: np.array([0,0,0,0,0.5,0.25,0.25,0.5,*(-ls.mean(axis=0) + init_ls)])
INIT_MOUNTPOINTS = np.array([[W, 0], [W, H], [0, H], [0, 0]]).T.flatten()

# Helper functions
def l_corr(ls, params):
    params = params.reshape(-1, 4)
    # return params[0] * np.square(ls) + (1 + 0.05 * np.tanh(params[1])) * ls + params[2]
    return params[0] * np.square(ls) + ls * params[1] + params[2]
    return np.sqrt(np.square(ls*params[1] + params[2]) - params[0])
def ik(x, mountPoints):
    mountPoints = mountPoints.reshape(1, 2, 4)
    mountPoints[0, :, 3] = [0, 0]
    # mountPoints[0, :, 0] = [W, 0]
    mountPoints[0, 1, 0] = 0
    mountPoints[0, 0, 2] = 0
    mountPoints = INIT_MOUNTPOINTS.reshape(1,2,4)
    return np.sqrt(np.sum(np.square(x.reshape(-1, 2, 1) - mountPoints), axis=1))
def err(ls, params):
    N = ls.shape[0]
    mountPoints = params[:8]
    lparams = params[8:20]
    xs = params[20:20 + 2 * N].reshape(-1, 2)
    return (l_corr(ls, lparams) - ik(xs, mountPoints)).flatten()

# Actual Calibrate Function Call
def calibrate(ls, init_params = None):
    if init_params is None:
        init_params = np.concatenate((INIT_MOUNTPOINTS, INIT_LPARAMS(ls), INIT_XS(ls)))
    return scipy.optimize.least_squares(lambda params: err(ls, params),
                                        init_params,
                                        verbose=2,
                                        method='lm')

# Do the calibration
# for every in (1000, 500):
for every in (1000, 500, 100):
    INIT_XS = lambda ls: xy[::every, :].flatten()
    result = calibrate(ls[::every])
    print(result.success, result.message)
    INIT_MOUNTPOINTS = result.x[:8]  # hack to set initialization
    INIT_LPARAMS = lambda ls: result.x[8:20]
# result2 = calibrate(ls[::100])


Now plot the calibration results to check if it seems reasonable.  Mainly, check that the right-most sub-plot matches the path you moved the robot end effector.

In [ ]:
# Extract parameters and plot
mountPoints = result.x[:8].reshape(2, 4)
lparams = result.x[8:20].reshape(-1, 4)
xs = result.x[20:].reshape(-1, 2)
print('mount points:')
print(mountPoints)
print('lparams:')
print(lparams)

import matplotlib as mpl
mpl.rcParams['axes.titlesize'] = 22
mpl.rcParams['axes.labelsize'] = 18

plt.figure(figsize=(12,4))
plt.subplot(121)
plt.plot(ls, ':')
plt.plot(l_corr(ls, lparams), '-')
plt.title('Cable Lengths')
plt.legend([f'cable {i} uncalibrated' for i in range(4)] + [f'cable {i} calibrated' for i in range(4)])
plt.xlabel('Data Sample \# (roughly 2ms / sample)')
plt.ylabel('Cable Length (m)')
plt.subplot(122)
plt.plot(xs[:, 0], xs[:, 1])
plt.title('Estimated Trajectory')
plt.xlabel('x (m)')
plt.ylabel('y (m)')
plt.axis('equal');

## Upload to the robot
Now, we are going to send the calibration parameters to the robot.

First we print out what we're about to send, as a final sanity check.  The first array should be the pulley locations (frame dimensions) and the second array should be the cable length paramters.  For each of the 4 columns (cables), the 3 rows are the a, b, c parameters of a quadratic:
$$ l_{calibrated} = a l_{raw}^2 + b l_{raw} + c $$
$a$ should be roughly 0, $b$ should be roughly the diameter of the winch in inches (divide by 2 if the cable is doubled), and $c$ should be roughly the length of the cable (in meters) at the "home" position of the end-effector.  The `c54` and `c44` lines are just useful references for copy-pasting if you want to send the calibration parameters manually for some reason.

In [ ]:
# Output in a format that can be sent directly to the cable robot
CARRIAGE_WIDTH, CARRIAGE_HEIGHT = 0.22377, 0.22377
mountPoints2 = mountPoints * 1
mountPoints2[0, 0:2] += CARRIAGE_WIDTH
mountPoints2[1, 1:3] += CARRIAGE_HEIGHT
print(mountPoints2) # sanity check
# lparams2 = np.vstack((lparams[0], np.ones(4), lparams[1]))
lparams2 = lparams
print(lparams2) # sanity check
print('-'*40)
print('c54', *mountPoints2.T.flatten().tolist(), sep=',')
print('c44', *lparams2.T.flatten().tolist(), sep=',')
print('-'*40)
print('c54', *mountPoints2.T.flatten().tolist(), sep=',', end=';')
print('c44', *lparams2.T.flatten().tolist(), sep=',')

Next, we send the above calibration parameters over serial to the cable robot.

**Make sure the robot is NOT in position control mode (e.g. put in HOLD or ESTOP) before running this**

Ensure that we receive all 8 acknowledgements back from the cable robot:
> Setting winch 0 mount point to ...  
> Setting winch 1 mount point to ...  
> Setting winch 2 mount point to ...  
> Setting winch 3 mount point to ...  
> Setting winch 0 length correction parameters to ...  
> Setting winch 1 length correction parameters to ...  
> Setting winch 2 length correction parameters to ...  
> Setting winch 3 length correction parameters to ...  

(If not, just run this cell again until all 8 acknowledgements are printed)

In [ ]:
with CableRobot(print_raw=True, write_timeout=None, initial_msg='d10,100', port='/dev/tty.usbmodem103568503') as robot:
    robot.update()
with CableRobot(print_raw=True, write_timeout=None, initial_msg='d10,100', port='/dev/tty.usbmodem103568503', silent=False) as robot:
    robot.send('g6')
    s1 = 'c54,' + ','.join(map(str, mountPoints2.T.flatten()))
    s2 = 'c44,' + ','.join(map(str, lparams2.T.flatten()))
    robot.send(s1)
    robot.send(s2)
    for _ in range(50):
        robot.update()
        time.sleep(0.005)

Finally, send some useful default settings to the robot.

* `kKM300` sets the max tension to 300N
* `kd0.5` sets the position error threshold to auto-exit position tracking mode to 0.5m
* `xL[direction][distance]` sets the workspace limits.  Direction can be l/d/u/r for the 4 directions and distance is a distance in meters relative to the origin (use `xl[dir][val]` to be relative to frame dimensions instead).  You may find suitable limits by first going into joystick mode, making the limits super lax (e.g. `xLl0;xLr99`) and then moving the robot around to what you think are reasonable limits and reading on the messages what is the estimated x/y position.

It is recommended to use this cell instead of manually sending these commands over the HTML control panel because these settings need to be set separately for each of the 3 controllers running on the cable robot!!!

In [ ]:
# Optionally, send the limits of the canvas
with CableRobot(print_raw=True, write_timeout=None, initial_msg='d10,100', port='/dev/tty.usbmodem103568503') as robot:
    robot.update()
with CableRobot(print_raw=True, write_timeout=None, initial_msg='d10,100', port='/dev/tty.usbmodem103568503', silent=False) as robot:
    for i in range(3):
        robot.send(f'gs{i}')
        # These are reasonable limits for the cable robot in Klaus
        # # robot.send('xLl0.7')
        # robot.send('xLl1.0')
        # # robot.send('xLd0.85')
        # robot.send('xLd0.65')
        # # robot.send('xLr2.35')
        # robot.send('xLr1.8')
        # # robot.send('xLu1.7')
        # robot.send('xLu1.0')

        robot.send('kKM300')
        robot.send('kd0.5')

        robot.send('xLl0.75')
        robot.send('xLd0.5')
        robot.send('xLr2.0')
        robot.send('xLu1.6')
        for _ in range(50):
            robot.update()
            time.sleep(0.005)
    robot.send(f'gs0')